# Template v2.0

- Updated: May 17 2022
- Author: Matthew Stachyra

In [31]:
import os
import re
import cv2
import sys
import random
import pandas as pd
import numpy as np
from typing import List
from PIL import Image
from matplotlib import pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
sys.setrecursionlimit(10000)

In [220]:
#####################################################
#                                                   #
#                                                   #
# Core image partitioning algorithms                #
#                                                   #
#                                                   #
#####################################################

def imPartition(path, images, reference, splices, blackthresh=0.80, bminpixel=5, anomthresh=0.10):
    """returns partitioned images together with the path to the original image and a label (0 or 1) for 
    whether it is anomalous.

    :param path: path to where the images are.
    :param images: dataframe with the images.
    :param reference: dictionary of anomaly tags to reference images generated by buildReference().
    :param splices: list of splices according to which to splice the images to. Data has form 
                    [[(rowindex1, rowindex2), (colindex1, colindex2), ...]]
    :param blackthresh: float point value between 0 and 1, with default of 0.8, that specifies how pixels
                        need to be black to constitute a black image that is rejected.
    :param bminpixel: integer value, with default of 5, that specifies the ceiling value for what constitutes
                      a black pixel.
    :param anomthresh: float point value between 0 and 1, with default of 0.1, that specifies how much of an
                       image needs to contains anomalous pixels (i.e., is a pixel part of anomaly) to be 
                       labeled with a 1.
    :returns: list of tuples as (path, numpy array, label of 0 or 1 for whether anomaly is present)
    """
    # maps the part() method to every image in images. This part() method partitions images by the provided
    # splices and append images with a label (0 or 1 if anomalous) if they contain enough non-background 
    # (i.e., are less than 1-blackthresh black).
    return [*map(lambda x : part(path, x, reference, splices, blackthresh, bminpixel, anomthresh),
               [tup[1] for tup in list(images.itertuples())])]  # list of image names/paths


def part(path, im, ref, splices, bthresh=0.8, minbpixel=5, athresh=0.1):
    """returns labeled (0 - nonanomalous, 1 - anomalous) partitions of the inputted image.

    NOTE  parameters described in imPartition.
    """
    # termination condition:
    # the image doesn't return a proper tag or contains only the anomaly
    tag = getTag(im)
    if not tag or "anomaly_only_view" in im: return

    # part input image according to input slices, keeping those partitions that are not too
    # black and labeling them according to whether they are anomalous or not
    refim = ref[tag]
    npim  = toNP(path, im)
    return [labelPart(im, npim, refim, s[0], s[1], athresh)  # s[0] is the row tuple and s[1] is the column tuple
                      for s in splices
                      if checkPart(npim, s[0], s[1], bthresh, minbpixel)]


def buildReference(path, ims, minpixel=5):
    """returns dictionary of tag:reference pairs where tag is the "P/d/d" anomaly tag and the reference
       is an array with all 0s except for 1s where an anomaly is present at that pixel

    :param path: path to where the images are.
    :param ims: dataframe with the images.
    :param minpixel: integer value, with default of 5, that specifies the ceiling value for what constitutes
                     a black pixel.
    :returns: dictionary where keys are the string anomaly tag and values are numpy arrays with 1s at the pixel
              value where there is anomaly present.
    """
    # helper: builds reference image that "traces" the anomaly and returns
    # an array with 1s at pixels where anomaly is
    def build(reference):
        zeros = np.zeros((len(reference), len(reference[0])))
        for i in range(len(reference)):
            for j in range(len(reference[i])):
                if np.all(reference[i][j] > minpixel):
                    BFS(i, j, reference, zeros)
        return zeros


    # helper: identifies which pixels contain anomaly using breadth first search,
    # looping through every pixel until we find an anomalous one, then considering every
    # neighbor that is anomalous until none are left / all are visited
    # assumption: there is only one anomaly in each image
    def BFS(row, col, reference, zeros):
        if row > len(reference)-1 or col > len(reference[0])-1 or row <0 or col <0: return
        if zeros[row][col]==1: return

        pixel = reference[row][col]
        if np.all(pixel > minpixel):
            zeros[row][col] = 1
        else:
            return

        moves = [(1,1), (-1,-1), (1,-1), (-1,1), (1,0), (0,1), (-1,0), (0,-1)]
        for m in moves:
            BFS(row+m[0], col+m[1], reference, zeros)


    # assumption: there exists an "anomaly only view" version of every image that can be "traced" to
    # identify which pixels are anomalous versus not anomalous using the above build and BFS methods
    # for each tag, builds reference image from zeros array by running BFS on the first pixel
    # that is non black, converting 0s to 1s for all adjacent pixels that are non black
    references = {}
    for i in ims.iterrows():
        imname = i[1][0]
        tag = getTag(imname) if getTag(imname) else ""
        if tag and tag not in references.keys():
            func = lambda x: ("anomaly_only_view" in str(x)) and (tag in str(x))

            # grab the next image (as its name in the df) that is a full image and has the anomaly in
            # the correct position (i.e., there is a match on the anomaly tag that is part of the name)
            reference = next(filter(func, [tup[1]  # holds the image name, whereas tup[0] holds the index
                                           for tup
                                           in list(ims.itertuples())]))
            references[tag] = build(toNP(path, reference)[:, :, :3])
    return references


def createSplices(path, im, mode='square', dim=64, k=None):
    """returns list of splices according to which to partition the image to.

    :param path: directory where the images are.
    :param im: numpy array of the image.
    :param mode: the type of partition algorithm used, with 'square' as default, and 'variable' as an option.
                 'Square' partitions the image into dim by dim partitions provided the input dim evenly 
                 divides the image dimensions. 'Variable' 
    :param dim: integer value specifying the dimension value used for partitioning.
    :returns: a method call that performs the specified partition algorithm in the mode parameter.

    NOTE  can add new modes for splicing in the future easily here together with a method to perform the
          partition.
    """
    if mode == 'square' and len(toNP(path, im))%dim!=0:
        raise AttributeError("'dim' of %d does not evenly divide image dimension %d by %d" % (xdim, len(npim), len(npim[0])))
    if mode == 'square':
        return squareSplice(path, im, dim)
    if mode == 'variable':
        return variableSplice(path, im, dim, k)


def squareSplice(path, im, dim):
    """return list of square (dim by dim) splices for the images.

    :param path: directory where the images are.
    :param im: numpy array of the image.
    :returns: splices that can be iterated over to generate partitions; these splices have the form of a list
              lists with tuples where the tuples contain the row and column splices for the partition.
    """
    npim = toNP(path, im)
    return [[(r, r+dim),(c, c+dim)]
            for r in range(0, len(npim), dim)
            for c in range(0, len(npim), dim)]

def variableSplice(path, im, dim, k):
    """return list of 2-tuples with indices of partitions of image, where image is a numpy array,
    integer ydim is used to generate the row windows of the partitions, and integer k is the number
    of features to extract

    :param path: directory where the images are.
    :param im: numpy array of the image.
    :param dim: integer value specifying the dimension value used for partitioning.
    :param k: the 'variable' or number of edges to identify in the image, used to generate splices.
    :returns: splices that can be iterated over to generate partitions; these splices have the form of a list
              lists with tuples where the tuples contain the row and column splices for the partition.
    """
    # build kMaxDiffs to store the k largest differences between successive column sums of pixel values
    # assumptions: 
    # - large differences in column pixel sums represent edges (such as the outlines of cargo containers)
    # - no vertical feature occurs more often than every 5 pixels and the cargo is at least 5 pixels wide
    #   giving us a featureWidth of 5
    # algorithm: get sum of pixel values for each column. then get difference between column
    # i and i-1. the greatest differences represent the greatest changes from dark to light,
    # or from no feature to some feature in the image.
    featureWidth = 5
    batchSize    = 64
    npim         = toNP(path, im)
    npim         = npim[:,:,:3].sum(axis=2)  # drop alpha channel and sum RGB channels
    
    colValues = []
    for col in range(len(npim[0])):
        rsum, count = 0, 0
        for i in range(len(npim[:][col])):
            pixel = npim[i][col]
            if pixel > 30:
                count += 1
                rsum += pixel
        if count != 0:
            colValues.append(rsum/count)
        else:
            colValues.append(0)
  
    batchDiffs, maxDiffs, kMaxDiffs = [], [], []
    for i in range(1, len(colValues)):
        diff = abs((colValues[i] - colValues[i-1]))
        if len(kMaxDiffs) < k: kMaxDiffs.append((i, diff))  # 2-tuples as (index, difference)
        batchDiffs.append((i, diff))

        # algorithm: get max of last 5 col differences (i.e., batch 5 pixel sums using our feature width)
        if i%featureWidth==0:
            maxBatchDiff = sorted(batchDiffs, key=lambda tup : tup[1], reverse=True)[0]
            maxDiffs.append(maxBatchDiff)
            batchDiffs = []

        # for each batch, grab the largest difference and append to k max diffs.
        # if its larger than any, replacing the smallest. this per batch process also ensures
        # an O(n) sort rather than O(nlogn) sort.
        if i%batchSize==0:
            maxDiff = sorted(maxDiffs, key=lambda tup : tup[1], reverse=True)[0][1]
            if any(list(map(
                    lambda x : x[1] < maxDiff,
                    kMaxDiffs))):
                diffOnly                   = [x[1] for x in kMaxDiffs]
                minLargest                 = min(diffOnly)
                minLargestIndex            = list(diffOnly).index(minLargest)
                kMaxDiffs[minLargestIndex] = (i, maxDiff)
            maxDiffs = []

    # assumption: hinges in door are equidistant so we can set the new x to be the
    # dfference between the first two values
    # algorithm: create k+1 windows of new x dimension by old y dimension (the
    # parameter dim) using kLargestDiffs, which is a list of 2-tuples of form
    # (index, difference) with the k largest differences in the image.
    kMaxDiffs.sort(key=lambda x : x[0])  # sort by index
    print(kMaxDiffs)
    xdim       = int(kMaxDiffs[1][0] - kMaxDiffs[0][0])  # set uniform x dimension
    startIndex = kMaxDiffs[0][0]  # grab first index
    return [[(r, r+dim),(c, c+xdim)]
            for c in range(startIndex, xdim*(k), xdim)
            for r in range(0, len(npim), dim)]


def checkPart(im, rsplice, csplice, bthresh, bminpixel):
    """returns True if partition is OK to include (i.e., is not too black); False, otherwise.

    :param:
    :param:
    :param:
    :param:
    :param:
    :returns:
    :rtype:
    """
    xdim = rsplice[1] - rsplice[0]
    ydim = csplice[1] - csplice[0]
    part = im[rsplice[0]:rsplice[1], csplice[0]:csplice[1]][:, :, :3]

    # bratio is the ratio of pixels where the RGB value is greater than the inputted min pixel
    # value for black, giving is a measure of how "black" a pixel
    # this then returns whether the ratio is under the threshold, which if it is, means the
    # part is not too black to include in training
    bratio = sum([0 if np.all(part[r][c] > bminpixel) else 1  # check if each pixel is greater than min
                    for r in range(xdim)
                    for c in range(ydim)]) / (xdim * ydim)
    return bratio < bthresh


def labelPart(imname, im, ref, rsplice, csplice, athresh):
    """returns 3-tuple of form (name of partitioned image, numpy array of partitioned image, 0 if nonanomalous; else 1).

    :param:
    :param:
    :param:
    :param:
    :returns:
    :rtype:
    """
    xdim = csplice[1] - csplice[0]
    ydim = rsplice[1] - rsplice[0]
    part = im[rsplice[0]:rsplice[1], csplice[0]:csplice[1]][:, :, :3]
    
    # aratio is the sum of pixel values in the reference for this splice divided by
    # the total number of pixels, giving us how much of the splice is anomalous
    # ref[r][c] is 1 pixel in the part generated from the inputted row and col splice
    aratio = sum([0 if ref[r][c] == 0 else 1
                    for r in range(rsplice[0], rsplice[1])
                    for c in range(csplice[0], csplice[1])]) / (xdim * ydim)
    return (imname, part, 1) if aratio > athresh else (imname, part, 0)


def saveParts(partedIms, anompath, noanompath):
    """saves partitioned images to disk in up to 2 different locations for anomalous
       versus non-anomalous images.

    :param:
    :param:
    :param:
    :param:
    :returns:
    :rtype:
    """
    for i in range(len(partedIms)):
        for j in range(len(partedIms[i])):
            imname  = partedIms[i][j][0][:-4]
            imarray = partedIms[i][j][1]
            imlabel = partedIms[i][j][2]
            im      = Image.fromarray(imarray)
            impath  = anompath if imlabel==1 else noanompath
            name    = os.path.join(impath, imname + str(i) + "_" + str(j) + ".png")
            im.save(name)

In [221]:
#####################################################
#                                                   #
#                                                   #
# Utils                                             #
#                                                   #
#                                                   #
#####################################################

def underSamp(x0, x1, ratio=[4,1]):
    """return randomized subset of the larger array corresponding to the inputted ratio.

    NOTE  x0 has form [(numpy array, 0)]
    NOTE  x1 has form [(numpy array, 1)]
    NOTE  ratio has form [int1, int2] describing ratio to balance x0 and x1
    NOTE  return has form [[(numpy array, 0)], [(numpy array, 1)]], which is a list
          of lists where each has size corresponding to the inputted ratio.
    """
    whichtosubset = 0 if len(x0) > ((ratio[0] / sum(ratio)) * (len(x0) + len(x1))) else 1
    if whichtosubset==0:
        tosubset = x0
        returnas = x1
    else:
        tosubset = x0
        returnas = x1
    indicescount = len(x1) * ratio[0] if whichtosubset==0 else len(x0)//ratio[0]
    indices      = [i for i in range(len(tosubset))]
    rindices     = random.sample(indices, indicescount)
    return [tup
            for i, tup in enumerate(tosubset)
            if i in rindices], returnas


def getTag(im):
    """return substring of image path string that represents anomaly positions.

    NOTE  different image naming convention will require different tagging
    """
    pattern = re.compile("[pP]\d{2}")
    tag     = [split for split in im.split("_") if pattern.match(split)]
    return tag[0] if tag else ""


def storeIms(ims, directory, tag=None):
    """save numpy images as pngs locally.
    """
    for i in range(len(ims)):
        for j in range(len(ims[i])):
            name = directory + tag + "_" + str(i) + "_" + str(j) + ".png"
            im   = Image.fromarray(ims[i][j])
            im.save(name)


def getIms(path):
    """return pandas dataframe with paths of pngs.
    """
    return pd.DataFrame([im for im in os.listdir(path) if im[-3:]=='png'])


def getImTypes(ims):
    """get image types for use with method subset_imgs.
    """
    return list(set([row[0].split("_")[1] for _, row in ims.iterrows()]))


def toNP(path, im):
    """return image img as numpy array.
    """
    return np.array(Image.open(os.path.join(path, im)))


def openIm(npim):
    """display inline numpy image img.
    """
    plt.figure(figsize = (25,10))
    plt.imshow(npim)


def subsetIms(ims, imtype, leaveout=False, types=
              ['Shirts', 'Paper', 'Laptops', 'Cans', 'Bananas', 'Shoes',
               'Apples', 'Tires', 'AnomalyAbsent', '200', '750']):

    """get subset of images of some type (i.e., string like '200', 'AnomalyAbsent', or 'Apples').

    NOTE  constrains types we can subset by the default value for List[str] in method definition.
    """
    if imtype not in types: raise AttributeError
    return ims[ims[0].str.contains(imtype, case=False) != leaveout]


def getRefIm(imdf):
    """return an image path (name) for a full image that isn't just the anomaly.
    """
    return next(filter(lambda x : "anomaly_only_view" not in x, [tup[1] for tup in list(imdf.itertuples())]))

In [237]:
#####################################################
# NOTE  update below to features such as which image
#       type of image to partition on, the partition
#       algorithm, and file path locations.

DIR         = os.path.join(os.path.dirname(os.getcwd()))            
SUBTYPE     = 'Shoes'      # which subtype of images to grab, if any else ""
MODE        = 'variable'   # either 'square' (dim by dim splices) or 'variable' (based on detected edges)
K           = {'Shirts': 5, 
               'Paper': 5, 
               'Laptops': 4, 
               'Cans': 4, 
               'Bananas': 4, 
               'Shoes': 10,
               'Apples': 4, 
               'Tires': 3}
DIM         = 64  # dim for 'square' partition mode or y dim for 'variable' partition mode
PATH        = os.path.join(DIR, 'images')
ANOMPATH    = os.path.join(PATH, 'partitioned', MODE, 'anom', SUBTYPE)
NOANOMPATH  = os.path.join(PATH, 'partitioned', MODE, 'noanom', SUBTYPE)
ANOMFILE    = os.path.join(ANOMPATH, str(SUBTYPE + str(1) + ".npy"))
NOANOMFILE  = os.path.join(NOANOMPATH, str(SUBTYPE + str(0) + ".npy"))

In [239]:
#####################################################
# NOTE  below should just run without any tweaking;
#       all tweaking is done above.

#####################################################
#                                                   #
#                                                   #
# Prepare images                                    #
#                                                   #
#                                                   #
#####################################################

if not os.path.exists(NOANOMFILE) and not os.path.exists(ANOMFILE):
    imdf = getIms(PATH)
    
    # temporary hack to remove 'Shirts' images with 200 where anomaly isn't visible
    if SUBTYPE=='Laptops': imdf = imdf[~imdf[0].str.contains("200")]
        
    imdf      = subsetIms(imdf, SUBTYPE)
    reference = buildReference(PATH, imdf)
    refim     = getRefIm(imdf)  
    splices   = createSplices(PATH, refim, mode=MODE, dim=DIM, k=K[SUBTYPE])  
    partimgs  = imPartition(PATH, imdf, reference, splices)

    part0, part1 = [], []
    parts = []
    for p in partimgs:
        if isinstance(p, list):
            parts.append(p)
            for i in range(len(p)):
                if p[i][2]==0:
                    part0.append((p[i][1], p[i][2]))
                elif p[i][2]==1:
                    part1.append((p[i][1], p[i][2]))
    np.save(NOANOMFILE, part0, allow_pickle=True)
    np.save(ANOMFILE, part1, allow_pickle=True)
    saveParts(parts, ANOMPATH, NOANOMPATH)
    

#####################################################
#                                                   #
#                                                   #
# Build dataset                                     #
#                                                   #
#                                                   #
#####################################################

part0 = np.load(NOANOMFILE, allow_pickle=True)
part1 = np.load(ANOMFILE, allow_pickle=True)

part0, part1 = underSamp(part0, part1)                                    # 80:20 distribution, by default
xfunc = lambda x : (np.asarray(x[0], dtype="float") /                     # normalize and flatten
                    np.linalg.norm(np.asarray(x[0], dtype="float"))).flatten()
yfunc = lambda x : np.asarray(x[1])
X = np.concatenate((
            np.asarray(list(map(xfunc, part0))),
            np.asarray(list(map(xfunc, part1)))), axis=0)
y = np.concatenate((
            np.asarray(list(map(yfunc, part0))),
            np.asarray(list(map(yfunc, part1)))), axis=0)

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42)


## Example running above algorithms with SVM:

#### Results as of  May 22
- Working types: Shirts, Apples, Paper, Bananas, Tires, Shoes
- Non working types: Cans, Shirts
- Perfect classification: Apples, Tires, Bananas
--- 
#### Best confusion matrices by type 
Note: below confusion matrices are with grid search on a predefined hyperparameter grid above <br>

- Apples: <br>
[[29  0] <br>
 [ 0  7]] <br> 1.0 <br> {'C': 1000, 'degree': 4, 'gamma': 0.1, 'kernel': 'rbf'} <br><br>
- Shirts: <br>
[[11  0] <br>
 [ 3 0]] <br> 0.7857142857142857<br> {'C': 0.1, 'degree': 4, 'gamma': 1, 'kernel': 'rbf'} <br><br>
- Bananas: <br>
[[25  0] <br>
 [ 0  5]] <br> 1.0 <br> {'C': 100, 'degree': 4, 'gamma': 1, 'kernel': 'rbf'} <br><br>
- Tires:
<br>
[[5  0] <br>
 [ 0  1]] <br> 1.0 <br> {'C': 100, 'degree': 4, 'gamma': 1, 'kernel': 'poly'} <br><br>
- Shoes:
<br>
[[45  0] <br>
 [ 3  9]] <br> 0.9473684210526315 <br> {'C': 100, 'degree': 4, 'gamma': 1, 'kernel': 'rbf'} <br> <br>
- Laptops:
<br>
[[11  0] <br>
 [ 3  0]] <br> 0.7857142857142857 <br> {'C': 0.1, 'degree': 4, 'gamma': 1, 'kernel': 'rbf'} <br> <br>

In [240]:
#####################################################
#                                                   #
#                                                   #
# Support Vector Machine                            #
#                                                   #
#                                                   #
#####################################################

print("Results below for: ", SUBTYPE, "\n")
clf  = svm.SVC(kernel='linear')
ypred = clf.fit(Xtrain, ytrain).predict(Xtest)
print(classification_report(ytest, ypred))
print(confusion_matrix(ytest, ypred))
print(accuracy_score(ytest, ypred))


grid = {'C':[0.1,1,100,1000],
        'kernel':['rbf','poly', 'linear'],
        'degree':[4, 5, 6, 10, 15],
        'gamma': [1, 0.1, 0.01]}
grid = GridSearchCV(svm.SVC(), grid, refit = True)
grid.fit(Xtrain, ytrain)
print(classification_report(ytest, grid.best_estimator_.predict(Xtest)))
print(confusion_matrix(ytest, grid.best_estimator_.predict(Xtest)))
print(grid.score(Xtest, ytest))
print(grid.best_params_)

Results below for:  Shoes 

              precision    recall  f1-score   support

           0       0.79      1.00      0.88        45
           1       0.00      0.00      0.00        12

    accuracy                           0.79        57
   macro avg       0.39      0.50      0.44        57
weighted avg       0.62      0.79      0.70        57

[[45  0]
 [12  0]]
0.7894736842105263


/Users/matthewstachyra/.local/share/virtualenvs/DISC-tlMijFAs/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matthewstachyra/.local/share/virtualenvs/DISC-tlMijFAs/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matthewstachyra/.local/share/virtualenvs/DISC-tlMijFAs/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        45
           1       1.00      0.75      0.86        12

    accuracy                           0.95        57
   macro avg       0.97      0.88      0.91        57
weighted avg       0.95      0.95      0.94        57

[[45  0]
 [ 3  9]]
0.9473684210526315
{'C': 100, 'degree': 4, 'gamma': 1, 'kernel': 'rbf'}
